<a href="https://colab.research.google.com/github/Platinum-Dragon/Ore-detector-presentation/blob/main/Pixie_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Questions to Answer:

> 1. ## Are there any other features that can be useful to improving Brand Safe labeling?
2. ## data 2
3. ## next item

not block quote

* [ ] Make test criteria
* [ ] Download data and explore








## Install necessary packages and mount the data from Google Drive

In [41]:
# Install icecream for easy print debugging
!pip install icecream

# Install pytest testing framework
!pip install pytest

# Install boto3
!pip install boto3
!pip install moto

# Install pytorch
!pip install transformers



In [42]:
from google.colab import drive
drive.mount("/content/drive")

!cp /content/drive/MyDrive/configuration.py /content

import pandas as pd
import numpy as np
import json
import re
import os
from icecream import ic
import boto3
from configuration import YOUTUBE_API_KEY



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [43]:
%%file test_notebook.py
import pytest
from moto import mock_s3
from icecream import ic
from configuration import YOUTUBE_API_KEY
from s3_utilities import S3_Handler

# Test if key is being imported
def test_API_key_is_imported():
  assert YOUTUBE_API_KEY is not None

def test_youtube_API_query():
  pass

@mock_s3
def test_create_s3_resource():
  outbound_features = S3_Handler()
  assert outbound_features.resource_exists()

@mock_s3
def test_create_bucket():
  outbound_features = S3_Handler()
  outbound_features.create_bucket("test_bucket")
  assert outbound_features.bucket is not None

@mock_s3
def test_s3_bucket_exists():
  outbound_features = S3_Handler()
  bucket = outbound_features.create_bucket("test-bucket")
  assert outbound_features.bucket_exists("test-bucket") == True

@mock_s3
def test_upload_to_s3():
  outbound_features = S3_Handler()
  test_bucket = outbound_features.create_bucket("test-bucket")
  test_bucket.upload_to_bucket()




Overwriting test_notebook.py


In [44]:
!python -m pytest test_notebook.py

============================= test session starts ==============================
platform linux -- Python 3.7.12, pytest-3.6.4, py-1.10.0, pluggy-0.7.1
rootdir: /content, inifile:
plugins: typeguard-2.7.1
collected 6 items                                                              

test_notebook.py .....F                                                  [100%]

=================================== FAILURES ===================================
______________________________ test_upload_to_s3 _______________________________

    @mock_s3
    def test_upload_to_s3():
      outbound_features = S3_Handler()
      test_bucket = outbound_features.create_bucket("test-bucket")
>     test_bucket.upload_to_bucket()
E     AttributeError: 'NoneType' object has no attribute 'upload_to_bucket'

test_notebook.py:35: AttributeError
====================== 1 failed, 5 passed in 0.97 seconds ======================


In [45]:
%%file s3_utilities.py
import boto3
from configuration import AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY, UPLOAD_FILENAME

class S3_Handler:
  def __init__(self):
    self.s3 = boto3.resource("s3",
                             aws_access_key_id=AWS_ACCESS_KEY_ID,
                             aws_secret_access_key=AWS_SECRET_ACCESS_KEY)
    self.upload_filename = UPLOAD_FILENAME


  def resource_exists(self):
    if self.s3:
      result = True
    else:
      result = False

    return result


  def create_bucket(self, name):
    self.bucket = self.s3.create_bucket(Bucket=name)


  def bucket_exists(self, name):
    try:
      self.s3.meta.client.head_bucket(Bucket=name)
      result = True
    except ClientError:
      result = False

    return result
      

  def upload_to_bucket(self, name):
    if self.s3.bucket_exists(Bucket=name):
      self.bucket.upload_file(Bucket=name, Filename="", Key="")
    else:
      pass


Overwriting s3_utilities.py


## Let us explore our data

In [46]:
# Look at our data
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 300)

features_df = pd.read_csv("/content/drive/MyDrive/ml-youtube-features.csv")
# ic(features_df.shape)
# ic(features_df.columns)

labeled_data_df = pd.read_csv("/content/drive/MyDrive/ml-youtube.csv")
# ic(labeled_data_df.shape)

# Isolate our features for our baseline model
features_title_desc_df = features_df[['id', 'video_title', 'video_description']]

# The labeled data has duplicates, because rows are created for "why" a video
# is not brand safe. So there will be one row marked "true" and two more with
# "Tobacco and Vaping" and "Sensitive Matters" labels. We want to keep our
# training to just true or false.

# Drop duplicates and discard unnecessary columns
labeled_data_df.drop_duplicates(subset=['item_id'], inplace=True)
labeled_data_df = labeled_data_df[['item_id', 'answer']].copy()

# We need to ensure 1-to-1 correspondence between features and labels
inner_join_df = labeled_data_df.merge(features_title_desc_df, left_on='item_id', right_on='id', how='inner')

# Is our training dataset balanced?
# Count % of true vs. false
ic(inner_join_df['answer'].value_counts())

# Our training dataset is very imbalanced:
# 97,381 - true | 10,117 - false
# So we need a more balanced dataset on which to train
all_true_inner_join_df = inner_join_df[inner_join_df['answer'] == "true"]
all_false_inner_join_df = inner_join_df[inner_join_df['answer'] == "false"]
true_sample_inner_join_df =\
all_true_inner_join_df.sample(n=all_false_inner_join_df.shape[0], random_state=222)

# This is a balanced dataset with the number of "true"s being randomly sampled
# to match the number of "false"s
balanced_inner_join_df =\
pd.concat([all_false_inner_join_df, true_sample_inner_join_df])
ic(balanced_inner_join_df['answer'].value_counts())

# Each feature row corresponds to a label row. Now separated them into two dataframes
features_title_desc_df = inner_join_df[['id', 'video_title', 'video_description']].copy()
labels_df = inner_join_df[['item_id', 'answer']].copy()




ic| inner_join_df['answer'].value_counts(): true                        97381
                                            false                       10117
                                            Adult & Explicit Content        1
                                            Name: answer, dtype: int64
ic| balanced_inner_join_df['answer'].value_counts(): true     10117
                                                     false    10117
                                                     Name: answer, dtype: int64


In [47]:
# # Train a base model with BERT
# # Verify GPU availability
import tensorflow as tf

# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   ic("GPU device not found")
# else:
#   print("Found GPU at: {}".format(device_name))

# # BERT imports
import torch
# from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertConfig, BertModel, BertForSequenceClassification
from keras.preprocessing.sequence import pad_sequences
# from sklearn.model_selection import train_test_split
# from tqdm import tqdm, trange
# import io
# import numpy as np
# import matplotlib.pyplot as plt
# % matplotlib inline

# # specify GPU device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
# torch.cuda.get_device_name(0)


In [48]:
# Convert each column to string type and clean unwanted elements
features_title_desc_df['video_title'] =\
features_title_desc_df['video_title']\
.apply(lambda x: str(x))

features_title_desc_df['video_description'] =\
features_title_desc_df['video_description']\
.apply(lambda x: str(x))

# Combine title and description into one feature
features_title_desc_df['video_description'] =\
features_title_desc_df['video_title'] + " " +\
features_title_desc_df['video_description']

# Remove excess spaces
features_title_desc_df['video_description'] =\
features_title_desc_df['video_description']\
.apply(lambda x: re.sub(r"\s{2,}", " ", x))

# Convert true/false labels to 1, 0
labeled_data_df['answer'] =\
labeled_data_df['answer'].apply(lambda x: 1 if x == "true" else 0)
ic(labeled_data_df['answer'].dtypes)

# ic(features_title_desc_df.loc[1, 'video_title'])
# ic(features_title_desc_df.loc[1, 'video_description'])



ic| labeled_data_df['answer'].dtypes: dtype('int64')


dtype('int64')

In [49]:
# Only using the top 100 entries to get pipeline running
sentences = features_title_desc_df.loc[0:99, 'video_description']
labels = labeled_data_df['answer'].to_numpy()[:100]

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=True)
tokenized_texts = [tokenizer.tokenize(sent)[:512] for sent in sentences]
print ("Tokenize the first sentence:")
print (tokenized_texts[0])

Tokenize the first sentence:
['trei', '##no', 'do', 'ga', '##do', 'na', 'cat', '##inga', '-', 'campo', 'sitio', 'novo', '-', '25', '/', '02', '/', '2018', 'publicado', 'em', '25', '/', '02', '/', '2018', 'a', 'des', '##ped', '##ida', 'da', 'va', '##ca', 'flores', '##ta', 'https', ':', '/', '/', 'you', '##tu', '.', 'be', '/', 's', '##cce', '##g', '##f', '##cy', '##icy']


In [50]:
# Just to make sure our tokens are within bounds
tokenized_lengths = [len(tokenized_text) for tokenized_text in tokenized_texts]
ic(np.max(tokenized_lengths))
ic(np.mean(tokenized_lengths))
ic(np.min(tokenized_lengths))

ic| np.max(tokenized_lengths): 512
ic| np.mean(tokenized_lengths): 273.23
ic| np.min(tokenized_lengths): 13


13

In [51]:
# Set the maximum sequence length. 
MAX_LEN = 512
# Pad our input tokens
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [52]:

# Tokenize all of the sentences and map the tokens to their word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 512,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
# print('Original: ', sentences[0])
# print('Token IDs:', input_ids[0])



Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2217: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [53]:
from torch.utils.data import TensorDataset, random_split

# Combine training inputs into a TensorDataset
dataset = TensorDataset(input_ids, attention_masks, labels)

# Save off validation dataset
train_size = int(0.8 * len(dataset))
validation_size = len(dataset) - train_size
train_dataset, validation_dataset = random_split(dataset, [train_size, validation_size])



In [54]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 16

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            validation_dataset, # The validation samples.
            sampler = SequentialSampler(validation_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

## Now we will create the model

In [55]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-multilingual-cased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

RuntimeError: ignored

## Create the optimizer

In [ ]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )


## Create the scheduler

In [ ]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 4

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

## Function to calculate accuracy

In [ ]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

## Function to caluculate / display elapsed time

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


## PyTorch Training Loop

In [ ]:
import random
import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # In PyTorch, calling `model` will in turn call the model's `forward` 
        # function and pass down the arguments. The `forward` function is 
        # documented here: 
        # https://huggingface.co/transformers/model_doc/bert.html#bertforsequenceclassification
        # The results are returned in a results object, documented here:
        # https://huggingface.co/transformers/main_classes/output.html#transformers.modeling_outputs.SequenceClassifierOutput
        # Specifically, we'll get the loss (because we provided labels) and the
        # "logits"--the model outputs prior to activation.
        result = model(b_input_ids, 
                       token_type_ids=None, 
                       attention_mask=b_input_mask, 
                       labels=b_labels,
                       return_dict=True)

        loss = result.loss
        logits = result.logits

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            result = model(b_input_ids, 
                           token_type_ids=None, 
                           attention_mask=b_input_mask,
                           labels=b_labels,
                           return_dict=True)

        # Get the loss and "logits" output by the model. The "logits" are the 
        # output values prior to applying an activation function like the 
        # softmax.
        loss = result.loss
        logits = result.logits
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

In [ ]:
# Retrain model with new feature
# Compare for stastical significance
# Repeat
# Recommendation